In [1]:
from laptrack import LapTrack
from laptrack import data_conversion
from laptrack import datasets
import geff
import tempfile
import napari
from skimage.measure import regionprops_table
import pandas as pd
from pathlib import Path
import zarr
import numpy as np

tmp_path = Path(tempfile.gettempdir())
print(f"Temporary files will be stored in: {tmp_path}")
!open {tmp_path}

Temporary files will be stored in: /var/folders/pz/_dlzclyn4mjgvngf1dpfg_880000gn/T


# Example of reading / saving data from / to GEFF

This example illustrates how we can save the tracked data into a GEFF (Graph Exchange File Format).

## Calculate region properties 

In [2]:
image, labels = datasets.cell_segmentation()
regionprops = []
for frame, label in enumerate(labels):
    df = pd.DataFrame(regionprops_table(label, properties=["label", "centroid"]))
    df["frame"] = frame
    regionprops.append(df)
regionprops_df = pd.concat(regionprops).rename(
    columns={"centroid-0": "y", "centroid-1": "x", "label": "seg_id"}
)
coordinate_cols = ["y", "x"]
frame_col = "frame"
regionprops_df.head()

,seg_id,y,x,frame
0,1,137.033613,96.336134,0
1,2,145.068966,178.988506,0
2,3,123.091743,203.266055,0
3,4,62.716216,141.290541,0
4,5,32.976744,124.348837,0


In [22]:
viewer = napari.Viewer()
viewer.add_image(image, name="image")
viewer.add_labels(labels, name="labels")
viewer.add_points(
    regionprops_df[[frame_col] + coordinate_cols],
    name="centroids",
    properties=regionprops_df,
    size=5,
    face_color="red",
)

<Points layer 'centroids' at 0x1495b6720>

## Track with LapTrack

In [4]:
lt = LapTrack(
    metric="sqeuclidean",
    cutoff=15**2,
    splitting_metric="sqeuclidean",
    merging_metric="sqeuclidean",
    splitting_cutoff=15**2,
    merging_cutoff=15**2,
)

In [ ]:
regionprops_df.head()

,seg_id,y,x,frame
0,1,137.033613,96.336134,0
1,2,145.068966,178.988506,0
2,3,123.091743,203.266055,0
3,4,62.716216,141.290541,0
4,5,32.976744,124.348837,0


/Users/fukai/projects/laptrack/.venv/lib/python3.12/site-packages/pydantic/_internal/_validators.py:52: UserWarning: Temporal unit frame not in valid OME-Zarr units [None, 'attosecond', 'centisecond', 'day', 'decisecond', 'exasecond', 'femtosecond', 'gigasecond', 'hectosecond', 'hour', 'kilosecond', 'megasecond', 'microsecond', 'millisecond', 'minute', 'nanosecond', 'petasecond', 'picosecond', 'second', 'terasecond', 'yoctosecond', 'yottasecond', 'zeptosecond', 'zettasecond']. Reader applications may not know what to do with this information.
  v_list = validator(input_value)
/Users/fukai/projects/laptrack/.venv/lib/python3.12/site-packages/pydantic/_internal/_validators.py:52: UserWarning: Spatial unit px not in valid OME-Zarr units [None, 'angstrom', 'attometer', 'centimeter', 'decimeter', 'exameter', 'femtometer', 'foot', 'gigameter', 'hectometer', 'inch', 'kilometer', 'megameter', 'meter', 'micrometer', 'mile', 'millimeter', 'nanometer', 'parsec', 'petameter', 'picometer', 'teramet

: 

In [5]:
track_df, split_df, merge_df = lt.predict_dataframe(
    regionprops_df,
    coordinate_cols=coordinate_cols,
    frame_col=frame_col,
)

track_df.head()

,seg_id,y,x,frame,tree_id,track_id
0,1,137.033613,96.336134,0,0,0
1,2,145.068966,178.988506,0,1,1
2,3,123.091743,203.266055,0,2,2
3,4,62.716216,141.290541,0,3,3
4,5,32.976744,124.348837,0,4,4


## Save to GEFF

In [6]:
tree_geff = data_conversion.dataframes_to_geff_networkx(
    track_df,
    split_df,
    merge_df,
    frame_col=frame_col,
)

In [7]:
tree_geff.nodes[0]

{'frame': 0,
 'seg_id': np.float64(1.0),
 'y': np.float64(137.03361344537817),
 'x': np.float64(96.33613445378151),
 'tree_id': np.float64(0.0),
 'track_id': np.float64(0.0)}

In [8]:
zarr_path = Path(tmp_path) / "cell_segmentation.zarr"
ds = zarr.open(zarr_path, mode="w")
ds.create_dataset("image", data=image)
ds.create_dataset("labels", data=labels)

meta = geff.GeffMetadata(
    related_objects=[
        {"type": "image", "path": "../image"},
        {"type": "labels", "path": "../labels", "label_prop": "seg_id"},
    ],
    track_node_props={"lineage": "tree_id", "tracklet": "track_id"},
    directed=True,
)

geff.write_nx(
    tree_geff,
    zarr_path / "track.geff",
    metadata=meta,
    axis_names=[frame_col] + coordinate_cols,
    axis_types=["time", "space", "space"],
    axis_units=["frame", "px", "px"],
)
geff.validate(zarr_path / "track.geff")
!ls -lh {zarr_path}/track.geff/nodes/props/

/Users/fukai/projects/laptrack/.venv/lib/python3.12/site-packages/geff/networkx/io.py:142: UserWarning: Both axis lists and metadata provided. Overriding metadata with axis lists.
  axis_names, axis_units, axis_types = _get_graph_existing_metadata(
/Users/fukai/projects/laptrack/.venv/lib/python3.12/site-packages/pydantic/main.py:253: UserWarning: Temporal unit frame not in valid OME-Zarr units [None, 'attosecond', 'centisecond', 'day', 'decisecond', 'exasecond', 'femtosecond', 'gigasecond', 'hectosecond', 'hour', 'kilosecond', 'megasecond', 'microsecond', 'millisecond', 'minute', 'nanosecond', 'petasecond', 'picosecond', 'second', 'terasecond', 'yoctosecond', 'yottasecond', 'zeptosecond', 'zettasecond']. Reader applications may not know what to do with this information.
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)
/Users/fukai/projects/laptrack/.venv/lib/python3.12/site-packages/pydantic/main.py:253: UserWarning: Spatial unit px not in valid

total 0
drwxr-xr-x 4 fukai staff 128 Jul 23 16:45 frame
drwxr-xr-x 4 fukai staff 128 Jul 23 16:45 seg_id
drwxr-xr-x 4 fukai staff 128 Jul 23 16:45 track_id
drwxr-xr-x 4 fukai staff 128 Jul 23 16:45 tree_id
drwxr-xr-x 4 fukai staff 128 Jul 23 16:45 x
drwxr-xr-x 4 fukai staff 128 Jul 23 16:45 y


# Command line tool for tracking

## Tracking CSV coordinates

In [20]:
!laptrack track --help

usage: laptrack track [--csv_path CSV_PATH]
                      [--track_geff_path TRACK_GEFF_PATH] --output_path
                      OUTPUT_PATH [--metadata_path METADATA_PATH]
                      [--frame_col FRAME_COL] --coordinate_cols
                      [COORDINATE_COLS ...] [-h] [--metric METRIC]
                      [--cutoff CUTOFF]
                      [--gap_closing_metric GAP_CLOSING_METRIC]
                      [--gap_closing_cutoff GAP_CLOSING_CUTOFF]
                      [--gap_closing_max_frame_count GAP_CLOSING_MAX_FRAME_COUNT]
                      [--splitting_metric SPLITTING_METRIC]
                      [--splitting_cutoff SPLITTING_CUTOFF]
                      [--merging_metric MERGING_METRIC]
                      [--merging_cutoff MERGING_CUTOFF]
                      [--track_start_cost TRACK_START_COST]
                      [--track_end_cost TRACK_END_COST]
                      [--segment_start_cost SEGMENT_START_COST]
                      [--

In [9]:
regionprops_df.to_csv(tmp_path / "regionprops.csv", index=False)

In [10]:
meta = geff.GeffMetadata(
    axes=[
        {"name": "frame", "type": "time", "unit": "frame"},
        {"name": "y", "type": "space", "unit": "px"},
        {"name": "x", "type": "space", "unit": "px"},
    ],
    related_objects=[
        {"type": "image", "path": "../image"},
        {"type": "labels", "path": "../labels", "label_prop": "seg_id"},
    ],
    track_node_props={"lineage": "tree_id", "tracklet": "track_id"},
    directed=True,
)
with open(tmp_path / "metadata.json", "w") as f:
    f.write(meta.model_dump_json(indent=2))

In [21]:
!cat {tmp_path}/metadata.json

{
  "geff_version": "0.3.1.dev29+g80dba08",
  "directed": true,
  "axes": [
    {
      "name": "frame",
      "type": "time",
      "unit": "frame",
      "min": null,
      "max": null
    },
    {
      "name": "y",
      "type": "space",
      "unit": "px",
      "min": null,
      "max": null
    },
    {
      "name": "x",
      "type": "space",
      "unit": "px",
      "min": null,
      "max": null
    }
  ],
  "sphere": null,
  "ellipsoid": null,
  "track_node_props": {
    "lineage": "tree_id",
    "tracklet": "track_id"
  },
  "related_objects": [
    {
      "type": "image",
      "path": "../image",
      "label_prop": null
    },
    {
      "type": "labels",
      "path": "../labels",
      "label_prop": "seg_id"
    }
  ],
  "affine": null,
  "display_hints": null,
  "extra": {}
}

In [11]:
!rm -r {zarr_path}/tracks2.geff
!laptrack track --csv_path {tmp_path}/regionprops.csv \
    --metadata_path {tmp_path}/metadata.json \
    --output_path {zarr_path}/tracks2.geff \
    --coordinate_cols y x \
    --frame_col frame \
    --metric sqeuclidean \
    --cutoff 225 \
    --splitting_metric sqeuclidean \
    --splitting_cutoff 225 \

rm: cannot remove '/var/folders/pz/_dlzclyn4mjgvngf1dpfg_880000gn/T/cell_segmentation.zarr/tracks2.geff': No such file or directory
/Users/fukai/projects/laptrack/.venv/lib/python3.12/site-packages/pydantic/main.py:746: UserWarning: Temporal unit frame not in valid OME-Zarr units [None, 'attosecond', 'centisecond', 'day', 'decisecond', 'exasecond', 'femtosecond', 'gigasecond', 'hectosecond', 'hour', 'kilosecond', 'megasecond', 'microsecond', 'millisecond', 'minute', 'nanosecond', 'petasecond', 'picosecond', 'second', 'terasecond', 'yoctosecond', 'yottasecond', 'zeptosecond', 'zettasecond']. Reader applications may not know what to do with this information.
  return cls.__pydantic_validator__.validate_json(
/Users/fukai/projects/laptrack/.venv/lib/python3.12/site-packages/pydantic/main.py:746: UserWarning: Spatial unit px not in valid OME-Zarr units [None, 'angstrom', 'attometer', 'centimeter', 'decimeter', 'exameter', 'femtometer', 'foot', 'gigameter', 'hectometer', 'inch', 'kilometer'

## Tracking GEFF to find missing edges

Making a geff file with missing edges

In [12]:
geff_tree, meta = geff.read_nx(zarr_path / "tracks2.geff")
all_edges = list(geff_tree.edges)
for e in np.random.choice(len(geff_tree.edges), 20, replace=False):
    e2 = all_edges[e]
    geff_tree.remove_edge(e2[0], e2[1])
!rm -r {zarr_path}/tracks3.geff
geff.write_nx(
    geff_tree,
    zarr_path / "tracks3.geff",
    metadata=meta,
)

rm: cannot remove '/var/folders/pz/_dlzclyn4mjgvngf1dpfg_880000gn/T/cell_segmentation.zarr/tracks3.geff': No such file or directory


In [16]:
!rm -r {zarr_path}/tracks4.geff
!laptrack track --track_geff_path {zarr_path}/tracks3.geff \
    --output_path {zarr_path}/tracks4.geff \
    --coordinate_cols y x \
    --frame_col frame \
    --metric sqeuclidean \
    --cutoff 225 \
    --splitting_metric sqeuclidean \
    --splitting_cutoff 225 \

rm: cannot remove '/var/folders/pz/_dlzclyn4mjgvngf1dpfg_880000gn/T/cell_segmentation.zarr/tracks4.geff': No such file or directory
/Users/fukai/projects/laptrack/.venv/lib/python3.12/site-packages/pydantic/_internal/_validators.py:52: UserWarning: Temporal unit frame not in valid OME-Zarr units [None, 'attosecond', 'centisecond', 'day', 'decisecond', 'exasecond', 'femtosecond', 'gigasecond', 'hectosecond', 'hour', 'kilosecond', 'megasecond', 'microsecond', 'millisecond', 'minute', 'nanosecond', 'petasecond', 'picosecond', 'second', 'terasecond', 'yoctosecond', 'yottasecond', 'zeptosecond', 'zettasecond']. Reader applications may not know what to do with this information.
  v_list = validator(input_value)
/Users/fukai/projects/laptrack/.venv/lib/python3.12/site-packages/pydantic/_internal/_validators.py:52: UserWarning: Spatial unit px not in valid OME-Zarr units [None, 'angstrom', 'attometer', 'centimeter', 'decimeter', 'exameter', 'femtometer', 'foot', 'gigameter', 'hectometer', 'inc

## Tracking images by overlap

In [ ]:
!laptrack overlap_track --help

usage: laptrack overlap_track --labels_path LABELS_PATH
                              [--metadata_path METADATA_PATH] --output_path
                              OUTPUT_PATH [-h] [--metric METRIC]
                              [--cutoff CUTOFF]
                              [--gap_closing_metric GAP_CLOSING_METRIC]
                              [--gap_closing_cutoff GAP_CLOSING_CUTOFF]
                              [--gap_closing_max_frame_count GAP_CLOSING_MAX_FRAME_COUNT]
                              [--splitting_metric SPLITTING_METRIC]
                              [--splitting_cutoff SPLITTING_CUTOFF]
                              [--merging_metric MERGING_METRIC]
                              [--merging_cutoff MERGING_CUTOFF]
                              [--track_start_cost TRACK_START_COST]
                              [--track_end_cost TRACK_END_COST]
                              [--segment_start_cost SEGMENT_START_COST]
                              [--segment_end_cost SEG

In [18]:
!laptrack overlap_track --labels_path {zarr_path}/labels \
    --output_path {zarr_path}/tracks5.geff \

# Viewering in napari

In [23]:
import napari

viewer = napari.Viewer()

Drag and drop the `cell_segmentation.zarr/track.geff` file into the viewer to visualize the image and labels.